In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings("ignore")

# Step 1: Load data
data = pd.read_csv("injury_severity_data.csv")

# Step 2: Encode labels
data['severity'] = data['severity'].map({'Minor': 0, 'Moderate': 1, 'Severe': 2})

# Step 3: Prepare features and labels
X = data.drop('severity', axis=1)
y = data['severity']

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train with XGBoost and hyperparameter tuning
params = {
    'max_depth': [3, 4],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 150],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3]
}
grid = GridSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
                    params, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)

# Step 6: Evaluation
print("Best parameters:", grid.best_params_)
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Minor', 'Moderate', 'Severe']))

# Step 7: Live prediction
live_input = pd.DataFrame([{
    'heart_rate': 59,
    'spo2': 93,
    'temperature': 35.6,
    'age': 70,
    'has_asthma': 1,
    'has_diabetes': 0
}])

pred = grid.predict(live_input)[0]
severity_map = {0: 'Minor', 1: 'Moderate', 2: 'Severe'}
print("Predicted Injury Severity:", severity_map[pred])


Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 150, 'subsample': 0.7}
              precision    recall  f1-score   support

       Minor       1.00      1.00      1.00        15
    Moderate       1.00      1.00      1.00        29
      Severe       1.00      1.00      1.00        56

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

Predicted Injury Severity: Moderate
